# Hands-on

## Setup Envs

### Config DB Engine and Pandas

In [13]:
import mysql.connector
from mysql.connector import Error
g_dbconfig = {
  'user': 'root',
  'password': 'root1MYSQL',
  'host': '127.0.0.1',
  'database': 'pymysql',
  'raise_on_warnings': True
}

myengine = mysql.connector.connect(**g_dbconfig)

In [14]:
from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine

In [15]:
alchemyconfig = {
    'user': 'root',
    'passw': 'root1MYSQL',
    'host': '127.0.0.1',
    'port': 3306,
    'database': 'pandasdb'
}

alchemyURL= 'mysql+mysqlconnector://{user}:{passw}@{host}/{database}'.format(**alchemyconfig)

mydb = create_engine(alchemyURL, echo=False)
readengine = create_engine(alchemyURL, echo=False)

In [16]:
pd.read_sql_query("show tables", mydb)

,Tables_in_pandasdb
0,dealer
1,goods
2,goods_customer
3,goods_promotion
4,goods_sell
5,past_due
6,retail_cus_ids
7,retail_transactions
8,retailcusids
9,retailtransactions


### Config Path and Envs

In [17]:
from pathlib import Path

In [18]:
data_dir = r'C:\Codes\DataScience\Data\mysql\data_capstone'
p = Path(data_dir)
csvfiles = [file for file in filter(lambda f: f.is_file() and (f.suffix == '.csv'), p.iterdir())]
dirs = [dirname for dirname in filter(lambda f: f.is_dir(), p.iterdir())]

In [61]:
stockdata_dir = r'C:\Codes\DataScience\Data\mysql\data_capstone\stk'
pstock = Path(stockdata_dir)
txtfiles = [file for file in filter(lambda f: f.is_file() and (f.suffix == '.txt'), pstock.iterdir())]

In [19]:
dirs

[WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk')]

In [62]:
txtfiles

[WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181003.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181004.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181005.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181008.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181009.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181010.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181011.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181012.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181015.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181016.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181017.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mys

In [20]:
csvfiles

[WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/past_due.csv'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/session.csv'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/TRANSACTION.csv')]

## ETL in Action

### Loading

In [44]:
# if_exists : {‘fail’, ‘replace’, ‘append’}, default ‘fail’
def load_csv(files, action='fail', block=512):
    for file in files:
        tmp_df = pd.read_csv(str(file))
        try:
            tmp_df.to_sql(name=file.stem, con=mydb, index=False, if_exists=action, chunksize=block)
            print(f'write {file.stem} to mysql!')
        except:
            df = pd.read_sql_query(f"show tables like '{file.stem}'", readengine)
            if df.empty:
                print(f'failed to write "{file.stem}" to mysql!')
            else:
                print(f'{file.stem} exists, pass loading.')
            continue

In [45]:
load_csv(csvfiles)

write past_due to mysql!
session exists, pass loading.
write TRANSACTION to mysql!


C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'TRANSACTION' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [49]:
pd.read_sql_query("select count(*) as total_row from past_due;", readengine)

,total_row
0,143712


In [ ]:
for file in txtfiles:
    tmp_df = pd.read_csv(str(file))
    try:
        tmp_df.to_sql(name=file.stem, con=mydb, index=False, if_exists=action, chunksize=block)
        print(f'write {file.stem} to mysql!')
    except:
        df = pd.read_sql_query(f"show tables like '{file.stem}'", readengine)
        if df.empty:
            print(f'failed to write "{file.stem}" to mysql!')
        else:
            print(f'{file.stem} exists, pass loading.')

## Analysis

In [58]:
143712*0.3

43113.6

In [56]:
df = pd.read_sql_query("select * from past_due where rand()<0.3;", readengine)

In [57]:
df

,id,report_date,past_due
0,Acc10,1-Feb-03,No
1,Acc123,1-Feb-03,No
2,Acc14,1-Feb-03,No
3,Acc140,1-Feb-03,No
4,Acc153,1-Feb-03,No
...,...,...,...
43090,Acc5953,26-Dec-05,No
43091,Acc5627,27-Dec-05,No
43092,Acc5796,27-Dec-05,No
43093,Acc5891,27-Dec-05,No


# END